In [ ]:
# | default_exp _code_generator.app_generator

In [ ]:
# | export

from typing import *
import time
import json
from tempfile import TemporaryDirectory
from pathlib import Path

from yaspin import yaspin

from fastkafka_gen._components.logger import get_logger
from fastkafka_gen._code_generator.helper import CustomAIChat, ValidateAndFixResponse
from fastkafka_gen._code_generator.prompts import APP_GENERATION_PROMPT

In [ ]:
from fastkafka_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

def _validate_response() -> List[str]:
    return []

In [ ]:
# | export


def _save_app(contents: str, output_path: str) -> str:
    """Save the YAML-formatted asyncapi spec in the specified output path.

    Args:
        contents: Generated python code
        output_file: The path to save the generated code.
    """
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    output_file = f"{output_path}/application.py"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(contents)
    
    return output_file

In [ ]:
fixture_response = """
print("Hello World")
"""


with TemporaryDirectory() as d:
    output_path = f"{str(d)}/parent/child"
    expected_output_file = f"{output_path}/application.py"
    
    actual_output_file = _save_app(fixture_response, output_path)
    
    assert actual_output_file == expected_output_file, f"{actual_output_file=}, {expected_output_file=}"
    
    with open(actual_output_file, 'r', encoding="utf-8") as f:
        contents = f.read()
    print(contents)



print("Hello World")



In [ ]:
# | export

def generate_app(asyncapi_spec: str, output_path: str) -> Tuple[str, str]:
    """Generate code for the new FastKafka app from the validated plan
    
    Args:
        asyncapi_spec: The validated AsyncAPI spec generated from the user's application description
        output_path: The path where the validated spec AsyncAPI file is located
    Returns:
        The generated FastKafka code
    """
    # TODO: Generate code form the ASYNCAPI
    # TODO: Validate the generated code
    with yaspin(text="Generating FastKafka app...", color="cyan", spinner="clock") as sp:
        
        app_generator = CustomAIChat(user_prompt=APP_GENERATION_PROMPT)
        app_validator = ValidateAndFixResponse(app_generator, _validate_response)
        validated_app, total_tokens = app_validator.fix(asyncapi_spec)
        
        output_file = _save_async_api_spec(validated_async_spec, output_path)
        
        sp.text = ""
        sp.ok(" ✔ FastKafka app generated and saved at: {output_file}")
        return validated_app, total_tokens

In [ ]:
# | notest

fixture_spec = '''
asyncapi: 2.5.0
info:
  title: Change Currency
  version: 0.0.1
  description: "A FastKafka application which utilizes localhost, staging, and production brokers creates personalized greetings. It consumes JSON-encoded messages containing product details, changes the currency and price based on specific conditions, and publishes the modified messages to a designated topic. It uses SASL_SSL with SCRAM-SHA-512 for authentication, requiring username and password."
  contact:
    name: Author
    url: https://www.google.com/
    email: noreply@gmail.com
servers:
  localhost:
    url: localhost
    description: local development kafka broker
    protocol: kafka
    variables:
      port:
        default: '9092'
  staging:
    url: staging.airt.ai
    description: staging kafka broker
    protocol: kafka-secure
    security:
    - staging_default_security: []
    variables:
      port:
        default: '9092'
  production:
    url: prod.airt.ai
    description: production kafka broker
    protocol: kafka-secure
    security:
    - production_default_security: []
    variables:
      port:
        default: '9092'
channels:
  store_product:
    subscribe:
      message:
        $ref: '#/components/messages/Product'
      description: |
        For each consumed message, check if the currency attribute is set to 'HRK'. If it is, change the currency to 'EUR' and divide the price by 7.5. If the currency is not set to 'HRK', do not modify the original message. Then, create a new object with the attributes country and store_product, set the country to 'IND' and the store_product to the modified message. Finally, publish the new message to the 'change_currency' topic.
  change_currency:
    publish:
      message:
        $ref: '#/components/messages/ModifiedProduct'
      description: Publish the modified message to the 'change_currency' topic.
components:
  messages:
    Product:
      payload:
        properties:
          product_name:
            description: Name of the product.
            title: Product Name
            type: string
          currency:
            description: Currency of the product.
            title: Currency
            type: string
            pattern: "^[A-Z]{3}$"
          price:
            description: Price of the product.
            title: Price
            type: number
        required:
        - product_name
        - currency
        - price
        title: Product
        type: object
    ModifiedProduct:
      payload:
        properties:
          country:
            description: Country of the modified message.
            title: Country
            type: string
          store_product:
            $ref: '#/components/messages/Product'
        required:
        - country
        - store_product
        title: Modified Product
        type: object
  schemas: {}
  securitySchemes:
    staging_default_security:
      type: scramSha512
    production_default_security:
      type: scramSha512
'''

code, token = generate_app(fixture_spec, spec_dir)
print(code)
print(token)